In [8]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [9]:
bart = torch.hub.load('pytorch/fairseq', 'bart.large.mnli')
bart.eval()

Using cache found in /home/henrike/.cache/torch/hub/pytorch_fairseq_main
2024-04-07 11:34:48 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.mnli.tar.gz from cache at /home/henrike/.cache/torch/pytorch_fairseq/5d221d78c74e1b8f4cad8f0e81927bd91d96f2f5da9a2547475e6212f5da3b89.4f1c48b34c2db97b55c12d62dace1b8776cfff21c8784b57a8047ff0f6389435


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/core/default_element.py:124: UserWarning: In 'config': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/1.2/upgra

BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_modul

In [10]:
dataset = pd.read_csv("../../ProSeCCo/fullCorpus/ProSeCCo_final.tsv", sep="\t")

In [11]:
locutions1 = list(dataset["locution1"])
locutions2 = list(dataset["locution2"])

propositions1 = list(dataset["proposition1"])
propositions2 = list(dataset["proposition2"])

In [12]:
all_loc_preds = []

for i in range(3):
    locution_preds = []

    for loc1, loc2 in zip(locutions1, locutions2):
        tokens = bart.encode(loc1, loc2)
        result = bart.predict("mnli", tokens).argmax()
        locution_preds.append(int(result))

    dataset["bart_mnli_locution_pred"] = locution_preds
    dataset.to_csv(f"bart-large-mnli-results/bart-large-mnli_locution-predictions_{i+1}.tsv", sep="\t", index=False)
    all_loc_preds.append(locution_preds)

In [13]:
all_prop_preds = []

for i in range(3):
    proposition_preds = []

    for prop1, prop2 in zip(propositions1, propositions2):
        tokens = bart.encode(prop1, prop2)
        result = bart.predict("mnli", tokens).argmax()
        proposition_preds.append(int(result))

    dataset["bart_mnli_proposition_pred"] = proposition_preds
    dataset.to_csv(f"bart-large-mnli-results/bart-large-mnli_proposition-predictions_{i+1}.tsv", sep="\t", index=False)

    all_prop_preds.append(proposition_preds)

In [14]:
dataset["bart_mnli_proposition_pred"] = proposition_preds
dataset.to_csv("bart-large-mnli-results/bart-large-mnli_proposition-predictions_2.tsv", sep="\t", index=False)

In [15]:
loc_accs = []
loc_f1s = []

prop_accs = []
prop_f1s = []

for i in range(1,4):
    locution_dataset = pd.read_csv(f"bart-large-mnli-results/bart-large-mnli_locution-predictions_{i}.tsv", sep="\t")
    proposition_dataset = pd.read_csv(f"bart-large-mnli-results/bart-large-mnli_proposition-predictions_{i}.tsv", sep="\t")

    loc_preds = [1 if lab==0 else 0 for lab in locution_dataset["bart_mnli_locution_pred"]]
    loc_gold = [1 if lab=="contradiction" else 0 for lab in locution_dataset["label"]]

    prop_preds = [1 if lab==0 else 0 for lab in proposition_dataset["bart_mnli_proposition_pred"]]
    prop_gold = [1 if lab=="contradiction" else 0 for lab in proposition_dataset["label"]]

    loc_accs.append(accuracy_score(loc_gold, loc_preds))
    loc_f1s.append(f1_score(loc_gold, loc_preds))
    
    prop_accs.append(accuracy_score(prop_gold, prop_preds))
    prop_f1s.append(f1_score(prop_gold, prop_preds))




In [18]:
print(loc_accs)
#prop_cm = confusion_matrix(prop_gold, prop_preds)

[0.5381132075471698, 0.5381132075471698, 0.5381132075471698]


In [17]:
from statistics import mean
import numpy as np

print("results with locutions:",
      "accuracy:", mean(loc_accs), "+-", np.std(np.asarray(loc_accs)), "\n",
      "f1:", mean(loc_f1s), "+-", np.std(np.asarray(loc_f1s))
)


print("results with propositions:",
      "accuracy:", mean(prop_accs), "+-", np.std(np.asarray(prop_accs)), "\n",
      "f1:", mean(prop_f1s), "+-", np.std(np.asarray(prop_f1s))
)

results with locutions: accuracy: 0.5381132075471698 +- 0.0 
 f1: 0.4354243542435424 +- 5.551115123125783e-17
results with propositions: accuracy: 0.550188679245283 +- 0.0 
 f1: 0.4649910233393178 +- 0.0
